# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [9]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [3]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [11]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer

In [4]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\user\Downloads\assignment5-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [5]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [6]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [7]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### step 1: cleaning the data:
in the section i delete from the "story" category all the English letters, numbers and Punctuation.

this step is implemented on both the train and the test sets.

at the end i separate the target category  from the train set and transfer the categories to numeric values.

In [8]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict
le = LabelEncoder()
global label
label = le.fit_transform(df_train['gender'])

In [9]:
def data_clean(train,test): 
    #transforn to list:
    train_list=train["story"].values.tolist()
    test_list=test["story"].values.tolist()
    
    #delete from train:
    for i,story in enumerate(train_list):
        train_list[i]=train_list[i].replace(',',"").replace(';',"").replace('.',"").replace(':',"").replace('!',"").replace('(',"").replace(')',"")
        train_list[i]=train_list[i].replace('"',"").replace("'","").replace("?","")
        for j in range (10):
            train_list[i]=train_list[i].replace(str(j),"")
        train_list[i] = re.sub(r"([A-Za-z]+)", '', train_list[i] )


    #delet from test:
    for i,story in enumerate(test_list):
        test_list[i]=test_list[i].replace(',',"").replace(';',"").replace('.',"").replace(':',"").replace('!',"").replace('(',"").replace(')',"")
        test_list[i]=test_list[i].replace('"',"").replace("'","").replace("?","")
        for j in range (10):
            test_list[i]=test_list[i].replace(str(j),"")
        test_list[i] = re.sub(r"([A-Za-z]+)", '', test_list[i] )


    #target category to numeric  and from list to df:
    train=train.drop("gender", axis=1)
    train = pd.DataFrame(train_list, columns=['story'])
    test=pd.DataFrame(test_list, columns=['story'])
    train["gender"] = label
    
    y=train["gender"]
    train=train.drop("gender", axis=1)

    return train,test,y

### step 2: machine learning:
in the section i implement the different machine modle learning.

for every model i search for the best hyper parameters in the different methods: count vectorize and tfidf.

In [23]:
#Decision Tree model:
def dt (method,df_train,df_test):
    train_df,test_df,y=data_clean(df_train,df_test)
    train_list=train_df["story"].values.tolist()
    if method=="c":
        vec=CountVectorizer(max_df=0.77)
        X_train = vec.fit_transform(train_list)
        X_train=pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
    if method=="t":
        vec = TfidfVectorizer(max_df=0.77)
        X_train= vec.fit_transform(train_list)
        X_train=pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
    
    param_grid = {'max_depth': [None,2, 5], 'min_samples_split': [2, 10,], 'min_samples_leaf': [1, 5, 10]}
    decisionTree = tree.DecisionTreeClassifier()
    grid_search = GridSearchCV(decisionTree, param_grid, scoring=make_scorer(f1_score, average='macro'), cv=7)
    grid_search.fit(X_train, y)
    decisionTree = tree.DecisionTreeClassifier(**grid_search.best_params_)

    # fit the tree with the traing data
    decisionTree = decisionTree.fit(X_train, y)
    # predict with the training data
    y_pred_train = decisionTree.predict(X_train)

    scorer_male = make_scorer(f1_score, pos_label=0)
    scorer_female = make_scorer(f1_score, pos_label=1)

    f1_scores_male = cross_val_score(decisionTree, X_train, y, cv=7, scoring=scorer_male)
    f1_scores_female = cross_val_score(decisionTree, X_train, y, cv=7, scoring=scorer_female)

    f1_scores_male_avg=sum(f1_scores_male) / len(f1_scores_male)
    f1_scores_female_avg=sum(f1_scores_female) / len(f1_scores_female)
    avg_total=(f1_scores_male_avg+f1_scores_female_avg)/2

    print("f1_macro:")
    print(avg_total)
    print("")
    print("f1:")
    print(f1_score(y, y_pred_train, average=None))

In [30]:
def calculating (method,model):
    train_df,test_df,y=data_clean(df_train,df_test)
    train_list=train_df["story"].values.tolist()
    if method=="c":
        vec=CountVectorizer(ngram_range=(1,2),min_df = 5, max_df=0.77)
        X_train = vec.fit_transform(train_list)
        X_train=pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
    elif method=="t":
        vec = TfidfVectorizer(ngram_range=(1,2),min_df = 5,  max_df=0.77)
        X_train= vec.fit_transform(train_list)
        X_train=pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
        
    if model=="Linear":
        param_grid = {'C': [0.1, 1, 10,50, 100], 'penalty': ['l1', 'l2']}
        clf = LinearSVC()
        grid_search = GridSearchCV(clf, param_grid, scoring=make_scorer(f1_score, average='macro'), cv=7)
        grid_search.fit(X_train, y)
        best_params = grid_search.best_params_
        clf = LinearSVC(C=best_params['C'])
    elif model=="knn":
        param_grid = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
        clf = KNeighborsClassifier()
        grid_search = GridSearchCV(clf, param_grid, scoring=make_scorer(f1_score, average='macro'), cv=7)
        grid_search.fit(X_train, y)
        best_params = grid_search.best_params_
        clf = KNeighborsClassifier(**best_params)
    elif model=="gnb":
        clf = GaussianNB() 
    elif model=="p":
        param_grid = {'penalty':[None,'l1', 'l2', 'elasticnet'],'alpha': [0.0001,0.05, 0.001, 0.01, 0.1],'max_iter': [1000, 5000, 10000]}
        clf = Perceptron()
        grid_search = GridSearchCV(clf, param_grid, scoring=make_scorer(f1_score, average='macro'), cv=7)
        grid_search.fit(X_train, y)
        best_params = grid_search.best_params_
        clf = Perceptron(**best_params)
    elif model=="MultinomialNB":
        clf = MultinomialNB()
    else: 
        if clf is None:
            print("Error: Model not recognized")
            return

    
    print("f1_macro:")
    scores = cross_val_score(clf, X_train, y,scoring=make_scorer(f1_score, average='macro'), cv=7)
    print(scores)
    print(f"Accuracy: {scores.mean()}")
    print("f1:")
    y_pred = cross_val_predict(clf, X_train, y, cv=7)
    print(f1_score(y, y_pred, average=None))

### step 3: comparing scores:
in the section i compar the scoring os each model and method to see what will be the best for the test set.

In [31]:
print("gnb- count vectorizer:")
print("---------------------")
calculating("c","gnb")
print("")
print("gnb- tfidf:")
print("-----------")
calculating("t","gnb")
print("")
print("******************************************")
print("knn- count vectorizer:")
print("---------------------")
calculating("c","knn")
print("")
print("knn- tfidf:")
print("-----------")
calculating("t","knn")
print("")
print("******************************************")
print("perceptron- count vectorizer:")
print("-----------------------------")
calculating("c","p")
print("")
print("perceptron- tfidf:")
print("------------------")
calculating("t","p")
print("")
print("******************************************")
print("decisionTree- count vectorizer:")
print("-------------------------------")
dt("c",df_train,df_test)
print("")
print("decisionTree- tfidf:")
print("--------------------")
dt("t",df_train,df_test)
print("******************************************")
print("LinearSVC- count vectorizer:")
print("----------------------------")
calculating("c","Linear")
print("")
print("LinearSVC- tfidf:")
print("-----------------")
calculating("t","Linear")
print("")
print("******************************************")
print("MultinomialNB- count vectorizer:")
print("--------------------------------")
calculating("c","MultinomialNB")
print("")
print("MultinomialNB- tfidf:")
print("---------------------")
calculating("t","MultinomialNB")



gnb- count vectorizer:
---------------------
f1_macro:
[0.47089947 0.42857143 0.54195095 0.50759878 0.43386243 0.43386243
 0.51257675]
Accuracy: 0.4756174641314317
f1:
[0.08510638 0.86949924]

gnb- tfidf:
-----------
f1_macro:
[0.47089947 0.42857143 0.54195095 0.50759878 0.43386243 0.43386243
 0.51257675]
Accuracy: 0.4756174641314317
f1:
[0.08510638 0.86949924]

******************************************
knn- count vectorizer:
---------------------
f1_macro:
[0.47640595 0.42553191 0.62105263 0.54621849 0.49074564 0.52232143
 0.69539337]
Accuracy: 0.5396670602004734
f1:
[0.256      0.85191083]

knn- tfidf:
-----------
f1_macro:
[0.60042283 0.57897567 0.625      0.64912281 0.64134078 0.56585518
 0.60709914]
Accuracy: 0.6096880593035259
f1:
[0.37414966 0.84818482]

******************************************
perceptron- count vectorizer:
-----------------------------
f1_macro:
[0.64872856 0.59298246 0.66168675 0.68727181 0.69560915 0.64825773
 0.6754988 ]
Accuracy: 0.6585764655103954
f1:
[

### step 4: test set:
in the section i implemented the best method and model on the test set to get a result.

in this case i chose count vectorizer on MultinomialNB.

In [32]:
train_list = df_train["story"].values.tolist()
vec = CountVectorizer(ngram_range=(1,2), min_df=5, max_df=0.75)
X_train = vec.fit_transform(train_list)
X_train = pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
y_train = df_train["gender"] # replace "target_variable" with the actual name of your target variable

clf = MultinomialNB()
scores = cross_val_score(clf, X_train, y_train, scoring=make_scorer(f1_score, average='macro'), cv=7)
clf.fit(X_train, y_train)

test_list = df_test["story"].values.tolist()
X_test = vec.transform(test_list)
X_test = pd.DataFrame(X_test.toarray(), columns=vec.get_feature_names_out())
y_pred = clf.predict(X_test)
df_test["gender"]=y_pred
df_test['gender'].value_counts()
df_predicted=df_test.copy()

MultinomialNB()

m    282
f     41
Name: gender, dtype: int64

In [33]:
df_predicted

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
...,...,...,...
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results.csv',index=False)